In [1]:
import numpy as np
import pandas as pd 

In [2]:
df = pd.read_csv("C:\\Users\\subha\\Downloads\\IMDB Dataset.csv")

In [3]:
df = df.iloc[:10000]

In [4]:
df.head(3)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


In [5]:
df['review'][1] 

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [6]:
df['sentiment'].value_counts()

positive    5028
negative    4972
Name: sentiment, dtype: int64

In [7]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [8]:
df.duplicated().sum()

17

In [9]:
df.drop_duplicates(inplace = True)

In [10]:
df.duplicated().sum() 

0

In [11]:
# Basic Preprocessing 
# Remove tags 
# Lowercase 
# remove stopwords 

In [12]:
import re 

def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '' , raw_text) 
    return cleaned_text 

In [13]:
df['review'] = df['review'].apply(remove_tags) 

In [14]:
df.head(2)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive


In [15]:
df['review'] = df['review'].apply(lambda x:x.lower()) 

In [16]:
from nltk.corpus import stopwords 
sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x : [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [17]:
df

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. filming technique...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive
...,...,...
9995,"fun, entertaining movie wwii german spy (julie...",positive
9996,"give break. anyone say ""good hockey movie""? kn...",negative
9997,movie bad movie. watching endless series bad h...,negative
9998,"movie probably made entertain middle school, e...",negative


In [18]:
x = df.iloc[: , 0:1]
y = df['sentiment']

In [19]:
x

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
9995,"fun, entertaining movie wwii german spy (julie..."
9996,"give break. anyone say ""good hockey movie""? kn..."
9997,movie bad movie. watching endless series bad h...
9998,"movie probably made entertain middle school, e..."


In [20]:
from sklearn.preprocessing import LabelEncoder 

encoder = LabelEncoder() 
y = encoder.fit_transform(y) 

In [21]:
from sklearn.model_selection import train_test_split 
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size =0.2 , random_state = 1) 


In [22]:
x_train.shape 

(7986, 1)

In [23]:
# Applying BOW 
from sklearn.feature_extraction.text import CountVectorizer 
cv = CountVectorizer()

In [24]:
x_train_bow = cv.fit_transform(x_train['review']).toarray() 
x_test_bow = cv.transform(x_test['review']).toarray() 

In [25]:
from sklearn.naive_bayes import GaussianNB 

gnb = GaussianNB() 
gnb.fit(x_train_bow , y_train) 

GaussianNB()

In [26]:
y_pred = gnb.predict(x_test_bow) 

from sklearn.metrics import accuracy_score , confusion_matrix 
# accuracy_score(y_test,y_pred) 

In [27]:
# confusion_matrix(y_test , y_pred)

In [28]:
from sklearn.ensemble import RandomForestClassifier 
# rf = RandomForestClassifier() 

# rf.fit(x_train_bow , y_train) 
# y_pred = rf.predict(x_test_bow) 
# accuracy_score(y_test , y_pred) 

In [29]:
cv = CountVectorizer(max_features = 3000)

x_train_bow = cv.fit_transform(x_train['review']).toarray() 
x_test_bow = cv.transform(x_test['review']).toarray() 

rf = RandomForestClassifier() 
rf.fit(x_train_bow , y_train)
y_pred = rf.predict(x_test_bow)
accuracy_score(y_test , y_pred)

0.8422633950926389

In [30]:
x_train_bow.shape 

(7986, 3000)

In [31]:
cv = CountVectorizer(ngram_range = (1,2) , max_features = 5000) 

x_train_bow = cv.fit_transform(x_train['review']).toarray() 
x_test_bow = cv.transform(x_test['review']).toarray() 

rf.fit(x_train_bow , y_train) 
y_pred = rf.predict(x_test_bow)
accuracy_score(y_test , y_pred)

0.8387581372058087

In [36]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7986 entries, 6713 to 235
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  7986 non-null   object
dtypes: object(1)
memory usage: 124.8+ KB


# using TFidf

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer 

In [33]:
tfidf = TfidfVectorizer()


In [34]:
x_train_tfidf = tfidf.fit_transform(x_train['review']).toarray() 
x_test_tfidf = tfidf.transform(x_test['review'])

In [35]:
rf = RandomForestClassifier() 

rf.fit(x_train_tfidf , y_train)
y_pred = rf.predict(x_test_tfidf)

accuracy_score(y_test , y_pred)

0.8372558838257386